# Simulated Annealing

Edoardo Bucheli A01016080
Ernesto Campos A00759359

In this notebook we present an implementation of the Simulated Annealing Algorithm to find the solution to a 2-dimensional Sensor Network problem. First we implement several helper functions and finally we present an example problem statement and the solution.

Let us first talk about the characterization of the problem. The terrain is represented as a 2D grid with dimensions $6\times 6$. We represent positions with a tuple $(a,b)$ where $a$ is the row and $b$ is the column. The upper left coordinate is $(1,1)$ and the bottom right corner is the coordinate $(6,6)$.

The terrain or `board` is represented as a list that is itself composed of lists each containing the information for a row. In its original configuration, a $1$ represents a cell that we wish to have a sensor in and $0$ represents a cell which does not interests us. This changes when a solution is generated, this is explained later in this document.

We use a list `sensors` whose length is equal to the number of sensors to place in the terrain. Each nth element `sensors[n]` represents the dimension of the receptive field of the sensor. So a sensor represented by the number $3$ has a receptive field of $3\times 3$. The coordinate given is the upper left corner of said recpetive field.

A solution is represented as a list of tuples specifying the position for each sensor. This must coincide with the list `sensors`, so the first element in a list `sol` refers to the position of the first element in the `sensors` list, and so on.

In [1]:
import numpy as np
import copy

The function `find_neighbors()` takes a coordinate from the terrain and returns a list with the adjacent cells to a given cell. This adjacent cells are in 8 directions, up, down, sides and diagonals. It takes into consideration edges and corners and thus will only return coordinates that are actually part of the grid.

In [2]:
def find_neighbors(loc):
    """Find neighbors of a position in the field"""
    neighbors = []
    if loc[0]-1<1 and loc[1]-1<1:
        neighbors.append((loc[0],loc[1]+1))
        neighbors.append((loc[0]+1,loc[1]))
        neighbors.append((loc[0]+1,loc[1]+1))
    elif loc[0]-1<1 and loc[1]+1>6:
        neighbors.append((loc[0],loc[1]-1))
        neighbors.append((loc[0]+1,loc[1]-1))
        neighbors.append((loc[0]+1,loc[1]))
    elif loc[0]+1>6 and loc[1]+1>6:
        neighbors.append((loc[0]-1,loc[1]-1))
        neighbors.append((loc[0]-1,loc[1]))
        neighbors.append((loc[0],loc[1]-1))
    elif loc[0]+1>6 and loc[1]-1<1:
        neighbors.append((loc[0]-1,loc[1]))
        neighbors.append((loc[0]-1,loc[1]+1))
        neighbors.append((loc[0],loc[1]+1))
    elif loc[0]-1<1:
        neighbors.append((loc[0],loc[1]-1))
        neighbors.append((loc[0],loc[1]+1))
        neighbors.append((loc[0]+1,loc[1]-1))
        neighbors.append((loc[0]+1,loc[1]))
        neighbors.append((loc[0]+1,loc[1]+1))
    elif loc[1]-1<1:
        neighbors.append((loc[0]-1,loc[1]))
        neighbors.append((loc[0]-1,loc[1]+1))
        neighbors.append((loc[0],loc[1]+1))
        neighbors.append((loc[0]+1,loc[1]))
        neighbors.append((loc[0]+1,loc[1]+1))
    elif loc[0]+1>6:
        neighbors.append((loc[0]-1,loc[1]-1))
        neighbors.append((loc[0]-1,loc[1]))
        neighbors.append((loc[0]-1,loc[1]+1))
        neighbors.append((loc[0],loc[1]-1))
        neighbors.append((loc[0],loc[1]+1))
    elif loc[1]+1>6:
        neighbors.append((loc[0]-1,loc[1]-1))
        neighbors.append((loc[0]-1,loc[1]))
        neighbors.append((loc[0],loc[1]-1))
        neighbors.append((loc[0]+1,loc[1]-1))
        neighbors.append((loc[0]+1,loc[1]))
    else:  
        neighbors.append((loc[0]-1,loc[1]-1))
        neighbors.append((loc[0]-1,loc[1]))
        neighbors.append((loc[0]-1,loc[1]+1))
        neighbors.append((loc[0],loc[1]-1))
        neighbors.append((loc[0],loc[1]+1))
        neighbors.append((loc[0]+1,loc[1]-1))
        neighbors.append((loc[0]+1,loc[1]))
        neighbors.append((loc[0]+1,loc[1]+1))
    return neighbors

The functions `print_board()` and `print_sol()` simply take a board and print it so it can be easily interpreted by the user. We use two functions since representation for a terrain and a solution is different. To understand the difference check the text before the function `make_sol()`

In [3]:
def print_board(board):
    """Print the initial board"""
    n = len(board)
    print('  ',end = '')
    for i in range(0,n):
        print(i+1,'',end ='')
    print('')
    for i in range(0,n):
        print(i+1,end = ' ')
        for j in range(0,n):
            if board[i][j] == 1:
                print('- ',end = '')
            else:
                print('X ',end = '')
        print('')

In [4]:
def print_sol(board):
    """Input a solution board as made by make_sol() and print it"""
    n = len(board)
    print('  ',end = '')
    for i in range(0,n):
        print(i+1,'',end ='')
    print('')
    for i in range(0,n):
        print(i+1,end = ' ')
        for j in range(0,n):
            print(board[i][j],'',end = '')
        print('')

### Characterization of a solution

There are several things we wish to represent in a solution, thus the representation for the configuration of a terrain and one where the sensors have been placed are different. 

A sensor may have a receptive field greater than 1 as specified by the `sensors` list. From the position coordinate, a sensor extends as many units to the right and below as its receptive field making a square. 

A number then, represents the sensor that is acting upon the cell. If a cell contains the number $1$ that means that that cell is being recorded by sensor 1. A 2 means the same for the second sensor and so on.

If a cell is within the receptive field of a sensor but it is a cell that we do not wish to record then a '#' symbol appears instead. If a cell is not within the receptive field of any sensor but we wish to record it, its value should be '-'. Finally, 'X' means that there is no sensor and we do not wish to record the cell anyway.

The formula `make_sol()` takes an original terrain configuration, a list that represents a solution `sol` and the list `sensors` and returns a board as described in this section.

In [5]:
def make_sol(board,sol,sensors):
    """
    Take board configuration, a solution and a list with the sensors and their size.
    Return board with sensors placed as specified by the solution and sensor size     
    """
    for i in range(0,6):
        for j in range(0,6):
            if board[i][j]==1:
                board[i][j]='-'
            elif board[i][j]==0:
                 board[i][j]='X'
    count = 1
    for n in sol:
        i = n[0]-1
        j = n[1]-1
            
        this_sensor = sensors[count-1]
        for k in range(0,this_sensor):
            for l in range(0,this_sensor):
                if not(i+k>5 or j+l>5):
                    if(board[i+k][j+l]=='-'):
                        board[i+k][j+l]=count
                    elif(board[i+k][j+l]=='X'):
                        board[i+k][j+l]= '#'
                else:
                    pass
        count += 1
    return board

### Fitness of a solution

We measure how good a solution is by checking how many of the desired cells are within the receptive field of a sensor. Therefore we wish to maximize our fitness.

In [6]:
def checksolution(board):
    """Check the fitness of a solution, how many of the desired cells have a sensor"""
    total = 0
    for i in range(0,6):
        for j in range(0,6):
            if type(board[i][j])==int:
                if board[i][j]>0:
                    total += 1
    return total

### Picking a new solution

Simulated Annealing requires us to generate new random or quasi-random states for each iteration. To do this we use the function pick random. It takes a solution (represented as a list of tuples just like before) and the list of sensors, mainly to know how many sensors a solution must consider. 

The function picks one of the sensors at random and extracts its current location. Then, one of the neighbors (obtained with `find_neighbors()`) is chosen at random and replaced to generate a new solution. The function `pick_random()` does this process and returns the index of the sensor to change and its new location.

In [7]:
def pick_random(sol,sensors):
    """Pick a random move from the list of possible ones"""
    n = np.random.choice(len(sensors))
    neighbors = find_neighbors(sol[n])
    choice = np.random.choice(len(neighbors))
    return n,neighbors[choice]

### Simulated Annealing

The function `simulated_annealing()` carries out the Simulated Annealing Algorithm. These are the basic steps.
1. If temperature is 0 return current solution
1. Pick a new solution at random as described before.
1. Compare the fitness of the previous solution and the new one.
1. If the new solution is better keep it.
1. If the previous solution is better keep the new solution with probability $e^{\Delta E/T}$. Where $\Delta E$ is the difference between the previous and the new solutions and T is the temperature.
1. Update Temparature.
1. Go back to step 1.

A lot of attention must be placed into how the temperature is initialized and updated. Bigger values of $T$ tend to need bigger updates to have a new probability that changes significantly, but smaller values rapidly decrease the probability if the reduction remains linear. Thus as time goes by, the algorithm reduces the amount by which the Temperature is reduced by dividing by the number of iterations, but once the reduction becomes less than $0.00001$ we make it steady to make sure it reaches 0, othewise it will go on forever.

One may notice that there are several commented lines in the code, this comments can be removed to see how the algorithm works when it runs, we have removed it to make a cleaner document but feel free to change if you wish to see how the algorithm manages the information. Beware, the algorithm does quite a bit of iterations in its current form.

In [8]:
def simulated_annealing(board,sol,sensors,T):
    """Carry out simulated annealing to solve the problem"""
    prev_sol = copy.deepcopy(sol)
    next_sol = copy.deepcopy(sol)
    iteration = 1
    while T > 0:
        
        #print("The temperature is: ",T)
        prev_board = copy.deepcopy(board)
        next_board = copy.deepcopy(board)
        
        next_sol = copy.deepcopy(prev_sol)
        
        choice = pick_random(prev_sol,sensors)
        next_sol[choice[0]]=choice[1]
        
        #print('Previous Solution: ',prev_sol)
        #print('Next Solution: ',next_sol)
        
        make_sol(prev_board,prev_sol,sensors)
        make_sol(next_board,next_sol,sensors)

        fit_prev = checksolution(prev_board)
        fit_next = checksolution(next_board)
        
        #print_sol(prev_board)
        #print("The Previous Fitness is: ",fit_prev)
        #print_sol(next_board)
        #print("The Next Fitness is:", fit_next)
        
        delta_e = fit_next-fit_prev
        #print("Delta_e ",delta_e)
        prob = np.exp(delta_e/T)
        if(delta_e>0):
            #print("Better!")
            prev_sol = next_sol
        else:
            #print(prob)
            choose = np.random.choice(2,p=[1-prob,prob])
            #print(choose)
            if choose:
                #print("Not better but prob: ",prob, "won")
                prev_sol = next_sol
            else:
                #print("Not Chosen")
                pass
                
        
        if 1/iteration>0.00001:
            T -= 1/iteration
        else:
            T -=0.00001
        iteration +=1
        #print('')
    #print(prev_sol)
    final_board = copy.deepcopy(board)
    make_sol(final_board,prev_sol,sensors)
    #print_sol(final_board)
    #print(checksolution(final_board))
    return prev_sol,final_board

### Experiment 

Let us show how this functions work first by showing the original board and then an example configuration.

Start by generating a board, a solution and a set of sensors.

In [9]:
board = [[1,1,0,0,1,0],[1,0,0,1,1,1],[0,0,1,1,1,1],[1,1,1,1,1,1],[0,0,0,1,1,1],[1,1,1,1,0,0]]

sol = [(1,1),(1,5),(4,5),(5,3)]

sensors = [3,2,2,1]

Let's see what this initial configutarion looks like.

In [10]:
print('The board without sensors looks like this: ')
print_board(board)
print('\nThe initial solution is: ')
initial_board = copy.deepcopy(board)
make_sol(initial_board,sol,sensors)
print_sol(initial_board)
print('It has a fitness of: ',checksolution(initial_board))

The board without sensors looks like this: 
  1 2 3 4 5 6 
1 - - X X - X 
2 - X X - - - 
3 X X - - - - 
4 - - - - - - 
5 X X X - - - 
6 - - - - X X 

The initial solution is: 
  1 2 3 4 5 6 
1 1 1 # X 2 # 
2 1 # # - 2 2 
3 # # 1 - - - 
4 - - - - 3 3 
5 X X # - 3 3 
6 - - - - X X 
It has a fitness of:  11


Now let's carry out Simulated Annealing to optimize the solution, we wish to maximize the fitness, which means more cells will have a sensor in it. 

In Simulated Annealing the choice of initial temperature and how it is updates is vital. Here we have chosen $T=10$.

In [11]:
solution,final_board = simulated_annealing(board,sol,sensors,T=10.0)

In [12]:
print('The solution found is:', solution)
print('With Fitness: ',checksolution(final_board))
print_sol(final_board)

The solution found is: [(3, 4), (1, 5), (3, 2), (1, 2)]
With Fitness:  16
  1 2 3 4 5 6 
1 - 4 X X 2 # 
2 - X X - 2 2 
3 X # 3 1 1 1 
4 - 3 3 1 1 1 
5 X X X 1 1 1 
6 - - - - X X 


If we run the algorithm several times we may find different solutions but utlimately the Fitness is allways 16 which is the best possible in this scenario.

### Second Experiment
Let's try to use the same algorithm with a different board and more sensors. The board we have created for this experiment has a pretty specific optimal solution and thus several possible local optima, let's see how the algorithm handles it.

In [13]:
board2 = [[1,1,1,1,1,0],[1,1,1,1,1,0],[0,0,1,1,1,0],[0,0,1,1,1,1],[1,1,0,1,1,1],[1,1,0,1,1,1]]

sol2 = [(1,1),(1,4),(4,1),(4,3),(6,1)]

sensors2 = [3,3,2,2,1]

In [14]:
print('The board without sensors looks like this: ')
print_board(board2)

The board without sensors looks like this: 
  1 2 3 4 5 6 
1 - - - - - X 
2 - - - - - X 
3 X X - - - X 
4 X X - - - - 
5 - - X - - - 
6 - - X - - - 


In [15]:
solution2,final_board2 = simulated_annealing(board2,sol2,sensors2,T=10.0)

In [16]:
print('The solution found is:', solution2)
print('With Fitness: ',checksolution(final_board2))
print_sol(final_board2)

The solution found is: [(4, 4), (1, 3), (5, 1), (1, 1), (4, 3)]
With Fitness:  27
  1 2 3 4 5 6 
1 4 4 2 2 2 X 
2 4 4 2 2 2 X 
3 X X 2 2 2 X 
4 X X 5 1 1 1 
5 3 3 X 1 1 1 
6 3 3 X 1 1 1 


We can see tha algorithm correctly finding the optimal solution where every desired cell has a sensor and we have a fitness of 27.